# Resumo relatório final

## Modelo Memory Polynomial (MP)

O modelo MP é descrito pela seguinte equação: 

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Onde $\tilde{out}$ e $\tilde{in}$ são vetores de valores complexos.



## Cálculo dos coeficientes

O cálculo dos coeficientes é feito como descrito na equação a seguir:

$$ \tilde{coefs}_{mp} = \frac{\tilde{out}_n}  {\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}} $$

Essa operação pode ser realizada utilizando a abordagem com matrizes complexas. 

In [2]:
import scipy.io as spio
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt


mat = spio.loadmat('data_LDMOS.mat')
in_ext = mat['in_extraction']
out_ext = mat['out_extraction']
in_val = mat['in_validation']
out_val = mat['out_validation']

P = 5
M = 2
n = 2


def x_mp(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    X_MP = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                c = ((m*P)-1)+p
                X_MP[r,c] = entrada[r-m,0]*((modulo_entrada[r-m,0])**(p-1))
    return X_MP

X_ext = x_mp(in_ext,M,P)
X_ext2 = X_ext[M+3:len(X_ext)-(M+3),:]
out_ext2 = out_ext[M+3:len(out_ext)-(M+3),:]
coefs = np.linalg.lstsq(X_ext2,out_ext2,rcond=-1)
coefs = coefs[0]
print(coefs)

[[ 1.2622929 -5.32196973e-03j]
 [-1.69747655+5.09301593e-02j]
 [ 3.45340987+7.74461073e-01j]
 [-3.30856233-2.26594244e+00j]
 [ 0.91061063+1.31286342e+00j]
 [-0.05662761+4.03202604e-04j]
 [ 0.32810271-1.40232176e-01j]
 [ 0.78543012+1.77217523e-01j]
 [-1.80250055-6.56437588e-02j]
 [ 0.95415195-4.09195545e-02j]
 [-0.04889592+8.81045717e-03j]
 [ 0.1645401 -3.24559864e-02j]
 [-0.83237918+1.70448069e-01j]
 [ 1.15547792-2.82329965e-01j]
 [-0.53874228+1.69059491e-01j]]


## Validação usando matrizes complexas:

Seguindo a equação:

$$\tilde{out}_n = \sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{coefs}_{mp}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$$

Para calcularmos a saída estimada para validação precisamos apenas multiplicar os coeficientes calculados anteriormente por $\sum^{P}_{p=1}\sum^{M}_{m=0}\tilde{in}_{n-m}|\tilde{in}_{n-m}|^{p-1}$


In [3]:
X_val = x_mp(in_val,M,P)
X_val2 = X_val[M+3:len(X_val)-(M+3),:]
out_calc_mat_cmplx = X_val2@coefs
print(out_calc_mat_cmplx)

[[0.17080977-0.45086617j]
 [0.09153475-0.46860405j]
 [0.00505675-0.46001652j]
 ...
 [0.13832978+0.07226911j]
 [0.1237369 +0.11137137j]
 [0.11426619+0.14626386j]]


### NMSE 

A métrica utilizada comparar o desempenho do modelo em cada abordagem será o NMSE, que é descrito pela seguinte equação:

$$ NMSE = 10log_{10}\frac{\sum^{N}_{n=1}|e_n|^2}{\sum^{N}_{n=1}|out_n|^2}$$

onde $out_n$ é o valor da amostra do sinal de saída no instante n, $e_n$ é o valor calculado
do erro entre as amostras do sinal de saída real e do sinal de saída simulado pelo modelo,
ou seja, $out_n − outcalc_n$ e N representa a quantidade total de amostras disponíveis.

In [4]:
out_val2 = out_val[M+3:len(X_val)-(M+3),:]
erro_mat_cmplx = out_val2 - out_calc_mat_cmplx
erro_mat_cmplx = np.sum(np.absolute(erro_mat_cmplx)**2)
out_ref_mat_cmplx = np.sum(np.absolute(out_val2)**2)
NMSE = 10*np.log10(erro_mat_cmplx/out_ref_mat_cmplx)
print('NMSE = ',NMSE)

NMSE =  -38.4445161722999


## Validação usando LUTS:

In [8]:
def in_abs(entrada, M, P):
    modulo_entrada = np.absolute(entrada)
    in_abs = np.zeros((len(entrada),P*(M+1)),dtype=complex)
    for r in range(M+1,len(entrada)):
        for m in range(M+1):
            for p in range(1,P+1):
                c = ((m*P)-1)+p
                in_abs[r,c] = (modulo_entrada[r-m,0])**(p-1)
    return in_abs


def lut(in_abs,M,P,n):
    xpol2 = in_abs[:,0:P-1]@coefs[0:P-1]
    for i in range(1,M+1):
        xpol = in_abs[:,i*P:((i*P)*(i+1))-1]@coefs[i*P:((i*P)*(i+1))-1]
        xpol2 = np.hstack((xpol2,xpol))

    xpol2 = xpol2[M+3:len(X_val)-(M+3),:]

    Q = 2**n
    z=np.linspace(0,1,Q)

    z = [int(round(a*(len(xpol2)-1))) for a in z]
    
    # lut = np.zeros((Q,(M+1)*2))
    # k = -1
    # for c in range(0,2*(M+1),2):
    #     k+=1
    #     for r in range(Q):
    #         lut[r,c] = np.real(xpol2[z[r],k])
    #         lut[r,c+1] = np.imag(xpol2[z[r],k])
    
    lut = np.zeros((Q,M+1),dtype=np.complex_)
    for c in range(0,M+1):
        for r in range(Q):
            lut[r,c] = xpol2[z[r],c]
    return lut




in_lut = in_abs(in_val,M,P)
lut = lut(in_lut, M, P, n)
print(lut)

[[ 0.87684764-0.05352152j  0.08443498-0.03310248j -0.05817805+0.00964452j]
 [ 0.96916181+0.01733429j  0.0608937 -0.02700679j -0.04379537+0.00759249j]
 [ 0.9668257 +0.01673899j  0.04485452-0.02435864j -0.04029612+0.00705578j]
 [ 1.04894257+0.01577934j  0.0089257 -0.01783284j -0.03932221+0.00691063j]]


In [11]:
from scipy import interpolate


def interpolacao(entrada,lut,M,n):
    Q = 2**n
    z = np.linspace(0,1,Q)

    abs_in = np.absolute(entrada)
    
    f = interpolate.interp1d(z,lut[:,0])
    inter = f(abs_in)
    for c in range(1,M+1):
        abs_in = np.absolute(in_val)
        f = interpolate.interp1d(z,lut[:,c])
        inter2 = f(abs_in)
        inter = np.hstack((inter,inter2))

    return inter

inter = interpolacao(in_val,lut,M,n)
inter = inter[M+3:len(X_val)-(M+3),:]

in_val_lut = in_val[M+3:len(X_val)-(M+3),:]

out_lut = np.multiply(inter,in_val_lut)
out_lut = out_lut.sum(axis=1)
out_lut = out_lut.reshape(len(out_lut),1)

erro_lut = out_val2 - out_lut
erro_lut = np.sum(np.absolute(erro_lut)**2)
out_ref_lut = np.sum(np.absolute(out_val2)**2)
NMSE_lut = 10*np.log10(erro_lut/out_ref_lut)
print(NMSE_lut)

-24.85407713520921


## Validação usando matrizes reais: